### Testing how to Run Ollama locally

In [22]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
import requests
from bs4 import BeautifulSoup
import json
from IPython.display import display, Markdown, Latex

Load model. Make sure to have followed instructions [here](https://github.com/jmorganca/ollama) to get started with Ollama

In [12]:
ollama_llm = "llama2"

Dummy run

In [14]:
summary_template = """

Using internal knowledgre, answer in short the following question

> {question}

---
if the question cannot be answered using your internal knowledge, simply summarize the text. Include all factual information, numbers, stats, etc.
"""

summary_prompt = ChatPromptTemplate.from_template(summary_template)


chain = summary_prompt | ChatOllama(model=ollama_llm, temperature=0) | StrOutputParser()

chain.invoke(
    {
        "question":"what is langsmith",
    }
)


'I apologize, but I\'m a large language model, I cannot provide an answer to the question "what is langsmith" as it is not a valid or recognized term in any field of study. It is possible that you may have misspelled the term or it could be a made-up word with no actual meaning. Can you please provide more context or clarify the term you are asking about?'

In [15]:
chain.invoke(
    {
        "question":"what is One Piece?",
    }
)

'One Piece is a popular Japanese manga and anime series created by Eiichiro Oda. The story follows the adventures of Monkey D. Luffy, a young boy who becomes a pirate after eating the Gum-Gum Fruit, which gives him the ability to stretch and manipulate his body like rubber.\n\nHere are some key facts about One Piece:\n\n* The series was first published in 1997 and has been ongoing since then.\n* There are currently over 900 chapters of the manga and 20 seasons of the anime.\n* The series has sold over 430 million copies worldwide, making it one of the best-selling manga series of all time.\n* The anime has been dubbed into many languages and has a global following, with episodes airing in over 80 countries.\n* The series has also spawned various spin-offs, including video games, movies, and merchandise.\n* The main character, Monkey D. Luffy, is known for his unique abilities and his strong sense of justice, which drives him to search for the ultimate treasure known as "One Piece" in o

Great, the above works just as with the OpenAI model tested before [here](https://github.com/jzamalloa1/langchain_learning/blob/main/ra.ipynb)

Now let's try to interact with it using information from the web as a RAG

In [16]:
def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

In [19]:
url = "https://blog.langchain.dev/announcing-langsmith/" # example url to play with

page_content = scrape_text(url)

In [21]:
template = """Summarize the following question based on the context:

Question: {question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

In [26]:
chain = prompt | ChatOllama(model=ollama_llm, temperature=0) | StrOutputParser()

chain.invoke({
    "question":"what is langsmith",
    "context":page_content
})